In [1]:
# Imports
import jupyterlab_dash
import os
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go

In [2]:
# Credits
# Inspiration from: https://github.com/plotly/simple-example-chart-apps/blob/master/plot_from_csv/apps/main.py

In [3]:
# Load and preprocess data

# read csv into dataframe
df = pd.read_csv('./data/HomeH-meter2_2016.csv')
# create additional time indices
df["month"] = pd.DatetimeIndex(df["Date & Time"]).month
df["date"] = pd.to_datetime(df['Date & Time'])

In [20]:
# Build App 
viewer = jupyterlab_dash.AppViewer()
app = dash.Dash()


######################
# LAYOUT
######################

app.layout = html.Div([
    html.Div([html.H1("Energy Consumption over Time")], style={'textAlign': "center"}),
    
    # SELECTOR FOR TRACES
    html.Div([dcc.Dropdown(id='value-selected',
                           options=[{'label': str(i), 'value': i} for i in df.columns.values[1:]],
                           # default trace
                           value=["Usage [kW]"],
                           multi=True,
                           style={"display": "block", "margin-left": "auto", "margin-right": "auto", "width": "80%"},
                           className="eight columns"),
              ],
             className="row"),
    
    # SLIDER FOR TIME RESAMPLING
     html.Div([
                            dcc.Checklist(
    id='checkbox-resampling',
    options=[{'label': 'Enabe resampling for total usage', 'value': 'true'}], value=['true']),
         
         dcc.Input(
    placeholder='',
    type='text',
    value='1'
),
         
         dcc.Slider(
                    id='my-slider',
                    min=1,
                    max=60,
                    step=1,
                    value=1,
                    marks={
        5: {'label': '5 min'},
        10: {'label': '10 min'},
        15: {'label': '15 min'},
        20: {'label': '20 min'},
        30: {'label': '30 min'},
        45: {'label': '45 min'},
        60: {'label': '60 min'},
    },
         
         ), 
                    html.Div(id='slider-output-container', style={"text-align": "center", "margin-top": "30px"}),
                   # RESET ZOOM ON UPDATE?
                    dcc.Checklist(
                    id='lock-zoom',
                    options=[{'label': 'Lock zoom', 'value': 'dataset'}], value=['dataset']),
                  html.Label('Mode of aggregation'),
    dcc.RadioItems(
        id='mode-of-resampling',
        options=[{'label': i, 'value': i} for i in ['Ommit', 'Sum', 'Average']],
        value='Ommit'
    ),

              
              
              ], 
              style={"display": "block", "margin-left": "auto", "margin-right": "auto", "width": "80%"},),
    


    # GRAPH
    dcc.Graph(id="my-graph"),
    
    # TIME RANGE SLIDE BELOW GRAPH
    html.Div([dcc.RangeSlider(id='month-selected', 
                              min=4, max=10, step=1,
                              marks={1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June", 7: "July", 8: "August", 9: "September",
                                     10: "October"}, value=[6, 8])]),
    
], className="container")

######################
# INTERACTIVITY
# called automatically on every input change
######################

# Slider
@app.callback(
    dash.dependencies.Output('slider-output-container', 'children'),
    [dash.dependencies.Input('my-slider', 'value')])
def update_output(value):
    
    return '\n Data will be resampled by {} minutes'.format(value)


@app.callback(
    dash.dependencies.Output("my-graph", "figure"),
    [dash.dependencies.Input("month-selected", "value"), 
     dash.dependencies.Input("value-selected", "value"), 
     dash.dependencies.Input('my-slider', 'value'),
     dash.dependencies.Input('lock-zoom', 'values'),
     dash.dependencies.Input('checkbox-resampling', 'values'),
     dash.dependencies.Input('mode-of-resampling', 'value')])
def update_graph(selected1, selected2, slidervalue, lockzoom, resample, mode):
    dff = df[(df["month"] >= selected1[0]) & (df["month"] <= selected1[1])]
    print(mode)
    if mode == "Ommit":
        df1 = df.resample(str(slidervalue) + 'min', on='date').first()
    elif mode == "Sum":
        df1 = df.resample(str(slidervalue) + 'min', on='date').sum()
    elif mode == "Average":
        df1 = df.resample(str(slidervalue) + 'min', on='date').average()
        
    dff1= df1[(df1["month"] >= selected1[0]) & (df1["month"] <= selected1[1])]
    
    new_list = []
    
    # selected values
    for i in selected2:
        new_list.append((str(i).replace('-', ' ')))
    
    # list of graphs
    trace = []
    
    

    # prepare traces according to selectors
    for indicator in selected2:
        trace.append(go.Scatter(x=dff['Date & Time'], y=dff[indicator], name=indicator, mode="lines",
                                marker={'size': 15, 'line': {'width': 0.5, 'color': 'white'}}, ))
    
    # 
    if resample == ['true']:
        trace.append(go.Scatter(x=dff1['Date & Time'], y=dff1['Usage [kW]'], name='Usage (resampled)', mode="lines",
                                marker={'size': 15, 'line': {'width': 0.5, 'color': 'white'}}, ))
        
    return {"data": trace, "layout": go.Layout(title=f"",
                                               xaxis={"title": "Date"}, yaxis={"title": f"Value"},
                                               colorway=["#C7037A", "#FFCB00", "#FF7C00", "#2F9609",
                                                         "#0E4770", "#A8AE0B"],
                                               # preserve zoom when updating dataset 
                                               # see: https://community.plot.ly/t/preserving-ui-state-like-zoom-in-dcc-graph-with-uirevision/15793 
                                               uirevision='dataset' if lockzoom == ['dataset'] else '')
           }


viewer.show(app)